# 3D Microstructure analysis (BETA)

::: warning
::: title
Warning
:::

Beta Features Disclaimer

-   This is beta documentation for one or more beta software features.
-   Beta features are considered unreleased and have not been fully
    tested nor fully validated. The results are not guaranteed by Ansys,
    Inc. (Ansys) to be correct. You assume the risk of using beta
    features.
-   At its discretion, Ansys may release, change, or withdraw beta
    features in future revisions.
-   Beta features are not subject to the Ansys Class 3 error reporting
    system. Ansys makes no commitment to resolve defects reported
    against beta features; however, your feedback will help us improve
    the quality of the product.
-   Ansys does not guarantee that database and/or input files used with
    beta features will run successfully from version to version of the
    software, nor with the final released version of the features. You
    may need to modify the database and/or input files before running
    them on other versions.
-   Documentation for beta features is called beta documentation, and it
    may not be written to the same standard as documentation for
    released features. Beta documentation may not be complete at the
    time of product release. At its discretion, Ansys may add, change,
    or delete beta documentation at any time.
:::

This example shows how to use PyAdditive to determine the
three-dimensional microstructure for a sample coupon with given material
and machine parameters.

Units are SI (m, kg, s, K) unless otherwise noted.


# Perform required import and connect

Perform the required import and connect to the Additive service.


In [ ]:
import pyvista as pv

from ansys.additive.core import (
    Additive,
    AdditiveMachine,
    Microstructure3DInput,
    SimulationError,
)

# Beta features must be enabled to use the 3D microstructure simulation.
additive = Additive(enable_beta_features=True)

# Select material

Select a material. You can use the
`~Additive.materials_list`{.interpreted-text role="meth"} method to
obtain a list of available materials.


In [ ]:
print("Available material names: {}".format(additive.materials_list()))

You can obtain the parameters for a single material by passing a name
from the materials list to the `~Additive.material`{.interpreted-text
role="meth"} method.


In [ ]:
material = additive.material("IN625")

# Specify machine parameters

Specify machine parameters by first creating an
`AdditiveMachine`{.interpreted-text role="class"} object and then
assigning the desired values. All values are in SI units (m, kg, s, K)
unless otherwise noted.


In [ ]:
machine = AdditiveMachine()

# Show available parameters
print(machine)

# Set laser power and scan speed

Set the laser power and scan speed.


In [ ]:
machine.scan_speed = 1  # m/s
machine.laser_power = 500  # W

# Specify inputs for 3D microstructure simulation

Specify microstructure inputs.


In [ ]:
input = Microstructure3DInput(
    machine=machine,
    material=material,
    sample_size_x=0.0001,  # in meters (.1 mm)
    sample_size_y=0.0001,
    sample_size_z=0.0001,
)

# Run simulation

Use the `~Additive.simulate`{.interpreted-text role="meth"} method of
the `additive` object to run the simulation. The returned object is
either a `Microstructure3DSummary`{.interpreted-text role="class"}
object or a `SimulationError`{.interpreted-text role="class"} object.


In [ ]:
summary = additive.simulate(input)
if isinstance(summary, SimulationError):
    raise Exception(summary.message)

# Plot 3D grain visualization

The `summary` object includes a VTK file describing the 3D grain
structure. The VTK file contains scalar data sets `GrainNumber`,
`Phi0_(deg)`, `Phi1_(deg)`, `Phi2_(deg)`, and `Temperatures`.


In [ ]:
# Plot the Phi0 data of the 3D grain structure
cmap = "coolwarm"
ms3d = pv.read(summary.grain_3d_vtk)
ms3d.plot(scalars="Phi0_(deg)", cmap=cmap)

# Add a cut plane to the plot
plotter = pv.Plotter()
plotter.add_mesh_clip_plane(ms3d, scalars="Phi0_(deg)", cmap=cmap)
plotter.show()

# Print average grain sizes

The `summary` object includes the average grain sizes in the XY, XZ, and
YZ planes.


In [ ]:
print("Average grain size in XY plane: {} µm".format(summary.xy_average_grain_size))
print("Average grain size in XZ plane: {} µm".format(summary.xz_average_grain_size))
print("Average grain size in YZ plane: {} µm".format(summary.yz_average_grain_size))